
# Day 08 Handheld Halting
[Link to Advent of Code 2020 - Day 09](https://adventofcode.com/2020/day/9)

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

- acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
- jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
- nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.
For example, consider the following program:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
These instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
```
First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

#### Part One

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

#### Part Two

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
```
After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

In [29]:
import copy

def run_program(boot_code: list) -> 'tuple(corruption_status: str, accumulator: int)':
    '''
    Purpose:
    - Runs through a boot_code list following the instruction of each element:
        - 'acc' -> increase or decrease the accumulator value by the arg value & move to next instruction
        - 'jmp' -> jump to another instruction in the list based on the arg value
        - 'nop' -> move on to next instruction
    Args:
    - boot_code list, which is a list of lists, each sub list contains:
        - operation: sting, the operation instruction, either 'nop', 'acc' or 'jmp'
        - arg: integer, a value to increase the accumulator by or to jump x number of instructions by, 
               can be positive or negative 
    Returns:
    - tuple containing:
        - infinite_loop_status: boolean, True if the program is an infinite loop, 
                                         False if the program moves outside of the instruction list
        - accumulator: integer, the value which is added to as the program is run through       
    '''
    accumulator = 0
    current_pos = 0
    visited_pos = []
    infinite_loop_status = True

    while current_pos not in visited_pos:
        if current_pos in range(len(boot_code)):
            infinite_loop_status = False
            break
        
        visited_pos.append(current_pos)
        operation, arg = boot_code[current_pos]
        
        if operation == 'acc':
            accumulator += arg
            current_pos += 1

        elif operation == 'jmp':
            current_pos += arg

        elif operation == 'nop':
            current_pos += 1
        
    return infinite_loop_status, accumulator

def break_infinite_loop(boot_code: list) -> 'tuple(accumulator, instruction, idx)':
    '''
    Purpose:
    - This function changes each 'nop' or 'jmp' element one at a time to see which singular change one will 
      break the infinite loop. 'nop' values are changed to 'jmp' and vice versa. 
    Args:
    - boot_code list, which is a list of lists, each sub list contains:
        - operation: sting, the operation instruction, either 'nop', 'acc' or 'jmp'
        - arg: integer, a value to increase the accumulator by or to jump x number of instructions by, 
               can be positive or negative 
    Returns:
    - tuple containing:
        - accumulator
        - instruction: list [operation, arg], the instruction that requires changing to break the infinite loop
        - idx: integer, index of the final changed instruction which breaks the infinite loop 
    '''
    for idx, instruction in enumerate(boot_code):
        # create a temporary boot_code list, deepcopy ensure no change to the original
        tmp_boot_code = copy.deepcopy(boot_code)    
        operation, _ = instruction

        # if the operation for this elemtn is either 'nop' or 'jmp' then....
        if operation in ('nop', 'jmp'):

            # change 'nop' to 'jmp' or 'jmp' to 'nop' at this index
            if   operation == 'nop': 
                tmp_boot_code[idx][0] = 'jmp'
            elif operation == 'jmp': 
                tmp_boot_code[idx][0] = 'nop'

            infinite_loop_status, accumulator = run_program(tmp_boot_code)

            if infinite_loop_status == False:
                idx, instruction
                break

    return (accumulator, instruction, idx)   


# opening input, splitting each line based on the space and converting the second element to an int
# with open('Inputs\day_08.txt') as file:        boot_code = [[line.split()[0], int(line.split()[1])] for line in file]
with open('Inputs\day_08_sample.txt') as file: boot_code = [[line.split()[0], int(line.split()[1])] for line in file]

_, pt1_acc = run_program(boot_code)
pt2_acc, pt2_inst, pt2_idx = break_infinite_loop(boot_code)

print(f'''
Part One: 
    The accumulator value is {pt1_acc} at the point when the program is about to repeat an instruction
Part Two: 
    The operation {pt2_inst} at index {pt2_idx} needs to change in order to break the infinite loop, 
    The final accumulator is: {pt2_acc}
''')


Part One: 
    The accumulator value is 0 at the point when the program is about to repeat an instruction
Part Two: 
    The operation ['nop', 0] at index 0 needs to change in order to break the infinite loop, 
    The final accumulator is: 0



In [33]:
help(break_infinite_loop)

Help on function break_infinite_loop in module __main__:

break_infinite_loop(boot_code: list) -> 'tuple(accumulator, instruction, idx)'
    Purpose:
    - This function changes each 'nop' or 'jmp' element one at a time to see which singular change one will 
      break the infinite loop. 'nop' values are changed to 'jmp' and vice versa. 
    Args:
    - boot_code list, which is a list of lists, each sub list contains:
        - operation: sting, the operation instruction, either 'nop', 'acc' or 'jmp'
        - arg: integer, a value to increase the accumulator by or to jump x number of instructions by, 
               can be positive or negative 
    Returns:
    - tuple containing:
        - accumulator
        - instruction: list [operation, arg], the instruction that requires changing to break the infinite loop
        - idx: integer, index of the final changed instruction which breaks the infinite loop



In [18]:
boot_code

[['acc', -5],
 ['nop', 333],
 ['acc', 45],
 ['jmp', 288],
 ['acc', -9],
 ['jmp', 1],
 ['acc', 27],
 ['jmp', 464],
 ['acc', 34],
 ['jmp', 478],
 ['jmp', 356],
 ['acc', 10],
 ['acc', 20],
 ['acc', 29],
 ['acc', -10],
 ['jmp', 359],
 ['acc', 29],
 ['acc', 31],
 ['acc', 36],
 ['acc', 42],
 ['jmp', 502],
 ['acc', 14],
 ['jmp', 45],
 ['jmp', 499],
 ['acc', -19],
 ['acc', 4],
 ['acc', 24],
 ['nop', 460],
 ['jmp', 465],
 ['acc', 29],
 ['acc', 6],
 ['acc', 25],
 ['jmp', 355],
 ['acc', -10],
 ['acc', 50],
 ['jmp', -27],
 ['acc', 46],
 ['acc', 2],
 ['acc', -18],
 ['acc', 8],
 ['jmp', 85],
 ['nop', 264],
 ['acc', 44],
 ['jmp', 310],
 ['acc', 23],
 ['acc', -15],
 ['acc', -12],
 ['jmp', 290],
 ['acc', -5],
 ['acc', 4],
 ['acc', -7],
 ['jmp', 248],
 ['acc', 23],
 ['jmp', 434],
 ['nop', -6],
 ['jmp', 239],
 ['jmp', 1],
 ['acc', -19],
 ['jmp', 67],
 ['acc', 40],
 ['acc', 21],
 ['acc', 24],
 ['jmp', 366],
 ['acc', 38],
 ['acc', 15],
 ['acc', -2],
 ['jmp', 542],
 ['acc', 27],
 ['acc', 21],
 ['acc', 44],
